In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
weather = pd.read_csv('data/min_weather.csv',encoding = 'cp949')
#peach = pd.read_csv('data/peach_produce.csv',encoding = 'cp949')
weather_copy = weather
#peach_copy= peach 


In [4]:
weather.info()
#each_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관측지점별   1410 non-null   object 
 1   시점      1410 non-null   float64
 2   평균기온    1410 non-null   float64
 3   평균최고기온  1410 non-null   float64
 4   평균최저기온  1410 non-null   float64
 5   최저기온    1410 non-null   float64
 6   합계강수량   1410 non-null   float64
 7   합계일조시간  1410 non-null   float64
dtypes: float64(7), object(1)
memory usage: 88.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시도별     20 non-null     object
 1   시점      20 non-null     int64 
 2   면적      20 non-null     object
 3   단위생산량   20 non-null     object
 4   생산량     20 non-null     object
dtypes: int64(1), object(4)
memory usage: 932.0+ bytes


In [ ]:
weather

In [31]:
#1. 시점 데이터를 연도와 월로 분리하기#



In [ ]:
#질문1 : to_datetime을 쓰면 년도가 시간으로 표현됨 
#weather['시점'] = pd.to_datetime(weather['시점'],format = '%Y.%m')  
# weather['year']= weather['시점'].dt.year
# weather['month']=weather['시점'].dt.month


In [7]:
weather_copy.insert(1,'year', weather['시점'].dropna().apply(lambda x:int(str(x)[0:4])))
weather_copy.insert(2,'month', weather['시점'].dropna().apply(lambda x:int(str(x)[5:7])))
#질문 : 10월은 1로만 나오는 이유  
#질문 : 10으로 바꾸는 방법 
weather[weather['month']==1]['month']=10


C:\Users\markp\AppData\Local\Temp\ipykernel_136324\1653601622.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather[weather['month']==1]['month']=10


In [5]:
#2. 월별 합계강수량, 합계일조시간을 월평균강수량, 월평균일조시간으로 바꾸기 
weather_copy['월평균강수량']=weather['합계강수량']/30
weather_copy['월평균일조시간']=weather['합계일조시간']/30
          

In [8]:
#2. 지점별, 연도별 data frame만들기
#기온 데이터 : 연평균기온, 겨울기온평균(12,1,2월), 연 최저기온
#강수량 데이터 : 연평균강수량, 생육기간평균강수랑(4~9월)
#일조량 데이터 : 연평균일조시간, 생욱기간(4~9월)일조량, 7월 일조량 )

year_data = weather_copy.groupby(['관측지점별','year'])['평균기온'].mean().to_frame('연평균기온')
year_data['겨울기온평균']= weather_copy[(weather.month==12) | (weather.month ==1)|(weather.month ==2) ].groupby(['관측지점별','year'])['평균기온'].mean()
year_data['최저기온'] =weather_copy[(weather.month==12) | (weather.month ==1)|(weather.month ==2)].groupby(['관측지점별','year'])['최저기온'].min()

year_data['연평균강수량'] = weather_copy.groupby(['관측지점별','year'])['월평균강수량'].mean().to_frame('연평균강수량')
year_data['4_9평균강수량'] = weather_copy[(weather.month>=4) & (weather.month <=9)].groupby(['관측지점별','year'])['월평균강수량'].mean()
year_data['6_9평균강수량'] = weather_copy[(weather.month>=6) & (weather.month <=9)].groupby(['관측지점별','year'])['월평균강수량'].mean()

year_data['연평균일조시간'] = weather_copy.groupby(['관측지점별','year'])['월평균일조시간'].mean().to_frame('연평균일조시간')
year_data['4_9평균일조시간']= weather_copy[(weather.month>=4) & (weather.month <=9)].groupby(['관측지점별','year'])['월평균일조시간'].mean()
year_data['6_9평균일조시간']= weather_copy[(weather.month>=6) & (weather.month <=9)].groupby(['관측지점별','year'])['월평균일조시간'].mean()
year_data['7월일조량']= weather_copy[weather_copy.month==7].set_index(['관측지점별','year'])['월평균일조시간']
year_data=year_data.reset_index()

In [9]:
year_data

,관측지점별,year,연평균기온,겨울기온평균,최저기온,연평균강수량,4_9평균강수량,6_9평균강수량,연평균일조시간,4_9평균일조시간,6_9평균일조시간,7월일조량
0,대전,2007,13.266667,5.350,-9.3,4.863611,8.279444,11.096667,5.318889,5.228889,4.036667,3.430000
1,대전,2008,12.975000,3.800,-11.6,2.882222,5.033333,6.770000,5.883056,5.939444,5.265000,3.543333
2,대전,2009,12.800000,4.400,-13.3,3.028889,4.831111,5.923333,6.056667,6.633333,6.030833,4.126667
3,대전,2010,12.650000,3.425,-14.4,3.943611,6.231667,7.630000,5.514722,5.652222,5.128333,4.336667
4,대전,2011,12.566667,2.500,-16.1,5.398333,9.577222,12.424167,5.690833,5.407222,4.804167,3.330000
...,...,...,...,...,...,...,...,...,...,...,...,...
113,충주,2017,12.025000,2.200,-16.0,2.980278,5.177222,6.902500,6.630000,7.105556,6.160833,3.733333
114,충주,2018,11.966667,0.525,-19.2,3.747500,5.771667,6.258333,6.523056,7.082222,7.108333,8.206667
115,충주,2019,12.300000,2.975,-13.2,2.209722,3.130000,4.079167,6.283056,6.638333,5.774167,4.613333
116,충주,2020,12.100000,2.875,-17.0,4.018056,6.955000,9.302500,6.172222,6.040000,5.003333,3.543333
